In [1]:
from pathlib import Path
import sys

import pandas as pd

sys.path.insert(0, str(Path(".").resolve().parent.parent))
from src import DATA_DIR

In [68]:
df = (
    pd.read_parquet(DATA_DIR / "lcl.snappy.parquet" / "part.2.parquet")
    .astype({"lclid": "string"})
    .drop_duplicates(["dt", "lclid", "c"])[["lclid", "dt", "c"]]
)

df

,lclid,dt,c
0,MAC000295,2012-05-01 21:30:00,0.456
1,MAC000295,2012-05-01 22:00:00,0.428
2,MAC000295,2012-05-01 22:30:00,0.425
3,MAC000295,2012-05-01 23:00:00,0.275
4,MAC000295,2012-05-01 23:30:00,0.351
...,...,...,...
936564,MAC000324,2012-05-19 03:30:00,0.012
936565,MAC000324,2012-05-19 04:00:00,0.037
936566,MAC000324,2012-05-19 04:30:00,0.069
936567,MAC000324,2012-05-19 05:00:00,0.012


In [69]:
df2 = (
    df.set_index([df.dt.dt.date, df.lclid, df.dt.dt.time])
    .drop(columns=["dt", "lclid"])
    .sort_index()
    .unstack()
)

df2.columns = df2.columns.droplevel(0)
df2.columns.name = None

# if you only want dt as the index
# df2 = df2.reset_index().set_index("dt")
# df2.index.name = None

df2

00:00:00  00:30:00  01:00:00  01:30:00  02:00:00  \
dt         lclid                                                         
2012-03-02 MAC000296       NaN       NaN       NaN       NaN       NaN   
           MAC000297       NaN       NaN       NaN       NaN       NaN   
           MAC000299       NaN       NaN       NaN       NaN       NaN   
2012-03-03 MAC000296     0.123     0.039     0.052     0.036     0.019   
           MAC000297     0.045     0.021     0.006     0.007     0.006   
...                        ...       ...       ...       ...       ...   
2014-02-28 MAC000319     0.367       NaN       NaN       NaN       NaN   
           MAC000320     1.161       NaN       NaN       NaN       NaN   
           MAC000321     0.291       NaN       NaN       NaN       NaN   
           MAC000322     0.137       NaN       NaN       NaN       NaN   
           MAC000323     0.334       NaN       NaN       NaN       NaN   

                      02:30:00  03:00:00  03:30:00  04:00:00  04:30:00  ...  \
dt         lclid                                                        ...   
2012-03-02 MAC000296       NaN       NaN       NaN       NaN       NaN  ...   
           MAC000297       NaN       NaN       NaN       NaN       NaN  ...   
           MAC000299       NaN       NaN       NaN       NaN       NaN  ...   
2012-03-03 MAC000296     0.020     0.067     0.020     0.020     0.023  ...   
           MAC000297     0.007     0.006     0.036     0.049     0.047  ...   
...                        ...       ...       ...       ...       ...  ...   
2014-02-28 MAC000319       NaN       NaN       NaN       NaN       NaN  ...   
           MAC000320       NaN       NaN       NaN       NaN       NaN  ...   
           MAC000321       NaN       NaN       NaN       NaN       NaN  ...   
           MAC000322       NaN       NaN       NaN       NaN       NaN  ...   
           MAC000323       NaN       NaN       NaN       NaN       NaN  ...   

                      19:00:00  19:30:00  20:00:00  20:30:00  21:00:00  \
dt         lclid                                                         
2012-03-02 MAC000296     0.281     0.214     0.786     1.064     0.856   
           MAC000297     0.046     0.045     0.017     0.006     0.007   
           MAC000299     0.949     0.582     0.630     0.173     0.118   
2012-03-03 MAC000296     0.434     0.338     0.339     0.218     0.199   
           MAC000297     0.937     0.881     0.848     0.147     0.132   
...                        ...       ...       ...       ...       ...   
2014-02-28 MAC000319       NaN       NaN       NaN       NaN       NaN   
           MAC000320       NaN       NaN       NaN       NaN       NaN   
           MAC000321       NaN       NaN       NaN       NaN       NaN   
           MAC000322       NaN       NaN       NaN       NaN       NaN   
           MAC000323       NaN       NaN       NaN       NaN       NaN   

                      21:30:00  22:00:00  22:30:00  23:00:00  23:30:00  
dt         lclid                                                        
2012-03-02 MAC000296     1.268     0.677     0.122     0.108     0.281  
           MAC000297     0.006     0.007     0.006     0.047     0.048  
           MAC000299     0.120     0.069     0.136     0.058     0.037  
2012-03-03 MAC000296     0.194     0.016     0.076     0.115     0.298  
           MAC000297     0.369     0.216     0.243     0.175     0.163  
...                        ...       ...       ...       ...       ...  
2014-02-28 MAC000319       NaN       NaN       NaN       NaN       NaN  
           MAC000320       NaN       NaN       NaN       NaN       NaN  
           MAC000321       NaN       NaN       NaN       NaN       NaN  
           MAC000322       NaN       NaN       NaN       NaN       NaN  
           MAC000323       NaN       NaN       NaN       NaN       NaN  

[19586 rows x 52 columns]

In [70]:
df3 = df2.copy()

df3 = df3.stack().reset_index()
df3["dt"] = pd.to_datetime(df3["dt"].astype(str) + " " + df3["level_2"].astype(str))
df3.drop(columns=["level_2"], inplace=True)
df3.rename(columns={0: "c"}, inplace=True)

df3

,dt,lclid,c
0,2012-03-02 11:30:00,MAC000296,0.054
1,2012-03-02 12:00:00,MAC000296,0.020
2,2012-03-02 12:30:00,MAC000296,0.059
3,2012-03-02 13:00:00,MAC000296,0.118
4,2012-03-02 13:30:00,MAC000296,0.078
...,...,...,...
935919,2014-02-28 00:00:00,MAC000319,0.367
935920,2014-02-28 00:00:00,MAC000320,1.161
935921,2014-02-28 00:00:00,MAC000321,0.291
935922,2014-02-28 00:00:00,MAC000322,0.137
